# Deep Convolutional Neural Networks with PyTorch

# Deep Neural Models in PyTorch


<span style="font-size:1.5em; color:blue">In this session, we will learn to use deep models provide by PyTorch from the `torchvision` package to create a classifier and perform cross validation.</span>

### ResNet50 

Reference [ResNet - Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)

![images/resnet50_kaggle.png MISSING](images/resnet50_kaggle.png)

**_Image from [Kaggle.com](https://kaggle.com)_**




In [ ]:
from pathlib import Path
from PIL import Image
import psycopg2
import torch
import torchvision
import torchvision.models as models

# Tranfer Learning with ResNet50

## Instantiating a model with new classifier

<span style="font-size:1.5em; color:blue">First, we create a pretrained model from `torchvision.models`.
Then we change the final fully connected layer with a `torch.nn.Linear` layer and specify the number of input/output features.</span>


In [ ]:
# Specify number of classes
NUM_CLASS = 45

def build_res50(fname_load=None):
    model = models.resnet50(pretrained=(fname_load is None))
    model.fc = torch.nn.Linear(in_features=2048, out_features=NUM_CLASS)
    if fname_load:
        print('Loading pretrained weights.')
        model.load_state_dict(torch.load(fname_load))
    return model

## Prepping Data Set for Training

<span style="font-size:1.5em; color:blue">In this stage we will prepare the dataset for cross validation.
<br/>
This is a little example that uses 3 classes with 200 images per class from RESISC45 dataset.
<br/>
We define a class which inherits from torch.utils.data.Dataset as an to load data for PyTorch models. 
<br/>
The `ImageDataset` class takes (image, class_label) as input metadata and loads the image from the disk as well.
<br/>
We have staged these metadata in a PostgreSQL database to facilitate cross validation, among other possible analyses, but we will also provide this metadata in a CSV format in the repository, so you do not have to rely on PostgreSQL.
</span>

In [ ]:
DATASET = '/dsa/data/IEEE/RESISC45'
METADATA = 'host=pgsql-ieee.dsa.lan dbname=resisc45 user=readonly_user password=fuzzieee'

In [ ]:
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, metadata, transform):
        self.dataset = Path(dataset)
        self.transform = transform
        self.metadata = metadata

    def __getitem__(self, idx):
        frame_id, image, class_label = self.metadata[idx]
        fname = str(self.dataset / image)
        return dict(frame_id=frame_id, image=self.transform(Image.open(fname).convert("RGB")), label=class_label)

    def __len__(self):
        return len(self.metadata)

In [ ]:
def xval_metadata(fold, test=False):
    print('Loading (traintest)', '=>', ('test' if test else 'train'))
    with psycopg2.connect(METADATA) as conn:
        with conn.cursor() as cur:
            if test:
                sql = """
                    select id, image, class_label from (select id, image, class_label, row_number()
                    over (partition by class_label order by random())
                    from frame where class_label in (3,8,33)) as foo
                    where row_number %% 5 = %s and row_number<201;
                """
            else:
                sql = """
                    select id, image, class_label from (select id, image, class_label, row_number()
                    over (partition by class_label order by random())
                    from frame where class_label in (3,8,33)) as foo
                    where row_number %% 5 != %s and row_number<201;
                """
            cur.execute(sql, (fold,))
            return cur.fetchall()

        

<span style="font-size:1.5em; color:blue">After each image is loaded a preprocessing pipeline can be supplied to PyTorch as well.
Here's an example, where we resize the input images and convert them into PyTorch tensors (from `PIL.Image`) and normalize.
</span>

> Some of the pipeline operations may only applied to tensors, others may be only applicable to images.
> This is also the reason we arrage the pipeline operations in such an order. Please refer to PyTorch documentation to find out more about the pipeline usage.
> ref: https://pytorch.org/docs/stable/torchvision/transforms.html

In [ ]:
transform_pipe = torchvision.transforms.Compose([
    torchvision.transforms.Resize(
        size=(299, 299)
    ),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

### Exporting metadata to .csv

In case you do not wish to rely on PostgreSQL, you may use the csv exports as metadata to initialize the `ImageDataset`. This is also available at `datafiles/*.csv`.

<span style="font-size:1.5em; color:blue">
To make use of this data, replace the xval_metadata() function and have it read from the CSVs.
 </span>

## Hyperparameters


<span style="font-size:1.5em; color:blue">Hyperparameters are the variables which determines the network and the training configuration,
e.g. units of neurons in each layer or the learning rate or the batch size.  
They are parameters that are not getting changed by the optimization algorithm,
and typically they remain constants throughout the network training.
</span>

In [ ]:
# Specify number of epochs per x-fold
#EPOCHS = 15
EPOCHS = 5

# Specify batch size for training and testing
BATCH_SIZE = 8

## Training

<span style="font-size:1.5em; color:blue">Below we are going to define the training loop for cross validation,
for each cross fold, where will be a training data set with 20% held out for validation.
The training and testing procedure within each cross validation iteration are the same as a standard train/test procedure, only with different data.</span>

First we create a little utility function to help progress reporting.

In [ ]:
from ipywidgets import IntProgress
from IPython.display import display
def progress_wrapper(iter_in, label='Fold Progress'):
    m = len(iter_in)-1
    pbar = IntProgress(description=label, min=0, max=m)
    display(pbar)
    for idx, item in enumerate(iter_in):
        pbar.value = idx
        yield item

In [ ]:
def train(model, epochs, fname_save=None, xval_fold=None, lr=1e-3, is_inception=False):
    train_data = ImageDataset(
        dataset=DATASET,
        metadata=xval_metadata(xval_fold, test=False),
        transform=transform_pipe
    )
    train_loader = torch.utils.data.DataLoader(
        train_data,
        batch_size=BATCH_SIZE,
        shuffle=True
    )
    test_data = ImageDataset(
        dataset=DATASET,
        metadata=xval_metadata(xval_fold, test=True),
        transform=transform_pipe
    )
    test_loader = torch.utils.data.DataLoader(
        test_data,
        batch_size=BATCH_SIZE
    )

    # optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()

    for i in range(epochs):
        model.train()
        ###################
        #     TRAINING    #
        ###################
        samples = 0
        loss_sum = 0
        true_sum = 0
        for batch in progress_wrapper(train_loader, 'train'):
            X = batch["image"]
            labels = batch["label"]
            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                if is_inception:
                    # ref: https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
                    y, aux_outputs = model(X)
                    loss1 = criterion(y, labels)
                    loss2 = criterion(aux_outputs, labels)
                    loss = loss1 + 0.4*loss2
                else:
                    y = model(X)
                    loss = criterion(y, labels)
                loss.backward()
                optimizer.step()
                loss_sum += loss.item() * X.shape[0]
                samples += X.shape[0]
                num_true = torch.sum(torch.argmax(y, 1) == labels)
                true_sum += num_true

        epoch_acc = float(true_sum) / float(samples)
        epoch_loss = float(loss_sum) / float(samples)
        print("epoch: {} - {} loss: {}, acc: {}".format(i + 1, "train", epoch_loss, epoch_acc))

        ###################
        #     TESTING     #
        ###################
        model.eval()
        samples = 0
        loss_sum = 0
        true_sum = 0
        for batch in test_loader:
            X = batch["image"]
            labels = batch["label"]
            with torch.set_grad_enabled(False):
                y = model(X)
                loss = criterion(y, labels)
                loss_sum += loss.item() * X.shape[0]
                samples += X.shape[0]
                num_true = torch.sum(torch.argmax(y, 1) == labels)
                true_sum += num_true

        epoch_acc = float(true_sum) / float(samples)
        epoch_loss = float(loss_sum) / float(samples)
        print("epoch: {} - {} loss: {}, acc: {}".format(i + 1, "test", epoch_loss, epoch_acc))

        ###################
        #     SAVING      #
        ###################
        if fname_save:
            torch.save(model.state_dict(), fname_save)

## Train - Test Methodolgies

### Cross-Validation Example

<span style="font-size:1.5em; color:blue">You should see training progress such as this</span> (except the result are misleading because we have extremely downsampled to accelerate training in order to take a screenshot):

![images/training.png](images/training.png)

In [ ]:
for fold in range(5):
    print("Fold", fold)
    model = build_res50()
    train(model, EPOCHS, xval_fold=fold, lr=1e-3, is_inception=False)

# Congrats, Part 2 complete!

**Appendix: extra source codes `src/`** 

* fusion_csv.sql - exporting cross validation metrics and inference data for the next data fusion stage
* metadata-resisc45.py - creating metadata for the RESISC45 dataset
* metadata-ucm.py - creating metadata for the UCMerced dataset
* tables.sql - metadata schemas definition
* train-resisc45.py - training ResNet50, Inception v3 and DenseNet on the RESISC45 dataset with cross validation
* train-ucm.py - training ResNet50, Inception v3 and DenseNet on the UCMerced dataset with cross validation